In [ ]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

In [ ]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

In [ ]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

In [ ]:
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary includeDistributionChart (summary:studies.Types.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    printfn $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss, {summary.StrategyName}"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
let signalDescription (signal:studies.Types.TradeOutcomeOutput.Row) =
    if signal.Gap > 0m && signal.Screenerid = 29 then
        "Gap ups - top gainers"
    else if signal.Gap > 0m && signal.Screenerid = 28 then
        "Gap ups - new highs"
    else if signal.Screenerid = 29 then
        "Top Gainers"
    else if signal.Screenerid = 28 then
        "New Highs"
    else if signal.Screenerid = 30 then
        "Top Losers"
    else if signal.Screenerid = 31 then
        "New Lows"
    else
        failwith $"Unexpected signal {signal.Screenerid}"
    
let filterNamePairs = [
        ("All", fun (o:studies.Types.TradeOutcomeOutput.Row) -> true)
        ("New Highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 28)
        ("Top Gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 29)
        ("Gap ups - new highs", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap > 0m && o.Screenerid = 28)
        ("Gap ups - top gainers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Gap > 0m && o.Screenerid = 29)
        ("New Lows", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 31)
        ("Top Losers", fun (o:studies.Types.TradeOutcomeOutput.Row) -> o.Screenerid = 30)
    ]

let createTradeSummaries outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Types.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    
                    let strategy = filteredTrades |> Seq.tryHead |> Option.map (fun x -> x.Strategy)
                    match strategy with
                    | None -> None
                    | Some strategy ->
                        let name = $"{filterName}, {strategy}"
                        filteredTrades |> studies.Types.TradeSummary.create name |> Some
            )
            |> Seq.choose id
    )
    

let describeOutcomes includeEVChart includeDistributionChart tradeSummaries =
    
    let groupedBySignal = tradeSummaries |> Seq.groupBy (fun (s:studies.Types.TradeSummary) -> s.StrategyName.Split(",")[0])
    
    groupedBySignal
    |> Seq.iter (
        fun (signal,summaries) ->
            
            printfn $"{signal} [{summaries |> Seq.head |> fun x -> x.Total}]"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart signal keys values
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
    
    let sortFunctions = [
        ("EV", fun (t:studies.Types.TradeSummary) -> t.EV)
        ("Win %", fun (t:studies.Types.TradeSummary) -> t.WinPct)
        ("Avg Gain", fun (t:studies.Types.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        tradeSummaries
        |> Seq.sortByDescending sortFunction
        |> Seq.take numberOfTopRecords
        |> Seq.iter (fun summary -> summary |> outputSummary false)
        printfn ""
        printfn ""
    )

In [ ]:
let studiesDirectory = "d:\\studies"
let inputFilename = "d:\\studies\\signals_transformed.csv"

let priceFunc = studies.DataHelpers.getPricesFromCsv studiesDirectory

let signals =
    inputFilename
    |> studies.Types.SignalWithPriceProperties.Load
    |> _.Rows
    
printfn "Loaded signals"

let strategies = [
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 5) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 5) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop false
    studies.Trading.strategyWithSignalCloseAsStop false
    studies.Trading.strategyWithTrailingStop false StockPositionType.Long 0.1m
    studies.Trading.strategyWithTrailingStop false StockPositionType.Short 0.1m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes |> Seq.length} trade outcomes"

printfn "Generating trade summaries..."
let tradeSummaries = outcomes |> createTradeSummaries |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

In [ ]:
// now, let's group by signal and see what we got
tradeSummaries
|> describeOutcomes true false

In [ ]:
tradeSummaries
|> List.filter (fun t -> t.EV > 0m)
|> List.sortByDescending (fun t -> t.EV)
|> describeOutcomes false false

In [ ]:
tradeSummaries
|> highlightStrategies 10

In [ ]:
let strategiesOfInterest = [
    "use trailing stop"
    "use signal open as stop"
    "use signal close as stop"
]

let summarizeSubset (name:string) =
    tradeSummaries
        |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
        |> Seq.sortByDescending (fun summary -> summary.EV)
        |> Seq.iter (fun summary -> 
            outputSummary false summary
        )
    
    printfn ""
    
strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)
    

In [ ]:
// sanity check, print random trades, look over them, investigate specific trades if needed

let outcomeDescription (x:studies.Types.TradeOutcomeOutput.Row) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> signalDescription
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened} -> {x.Closed}, {x.OpenPrice} -> {x.ClosePrice}: profit of {gain}, signal: {signal}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Sell"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Types.TradeOutcomeOutput.Row) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

In [ ]:
// investigating what happened with the trade and why it went as it went
let signals =
    inputFilename
    |> studies.Types.SignalWithPriceProperties.Load
    |> _.Rows

let tickerOfInterest = "MDGL"

let signalsOfInterest = 
    signals
    |> Seq.filter (fun signal -> signal.Ticker = tickerOfInterest)
    
let strategy = studies.Trading.strategyWithTrailingStop true StockPositionType.Long 0.1m

let outcomesOfInterest = studies.Trading.runTrades priceFunc signalsOfInterest [strategy] |> Async.RunSynchronously

outcomesOfInterest
|> Seq.iter ( fun outcome -> 
    outcome |> outcomeDescription
    printfn ""
)

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Types.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.Total)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    printfn $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    printfn $"mapped keys and values for {label}, charting..."
    
    values |> List.zip keys |> List.iter (fun (l,v) -> printfn $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    printfn ""
)

In [ ]:
// let's take only the trades where EV is positive
tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary false s)

In [ ]:
let benchmarkStrategyName = "Gap ups - new highs, B&H"

let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary false benchmarkStrategy